In [ ]:
!pip install praw
nltk.download('stopwords')


In [ ]:
import praw
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import Counter
from nltk.util import bigrams
import string
import warnings

warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id="uTp5HipGLhBhPv1S7oxk8A",
    client_secret="AcT-azqNdIT3onTltclkX4Q3ISUDDQ",
    user_agent="mads",
)

# Fetch top 100 posts from the 'netflix' subreddit from the past month
subreddit = reddit.subreddit('netflix')
top_posts = subreddit.top('month', limit=100)

# Define stopwords to be removed from analysis
stop_words = set(stopwords.words('english'))
custom_stopwords = {"netflix", "season", "new", "shows", "series", "show", "movie", "movies","streaming"}
stop_words = stop_words.union(custom_stopwords)

# Gather titles and comments
all_texts = []
for post in top_posts:
    all_texts.append(post.title)
    for comment in post.comments:
        if isinstance(comment, praw.models.MoreComments):
            continue
        all_texts.append(comment.body)
all_texts = " ".join(all_texts).lower()

# Tokenize and filter stopwords
tokens = [word.strip(string.punctuation) for word in all_texts.split()]
filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

# Create bigrams
bi_grams = list(bigrams(filtered_tokens))

# Count them
bi_gram_freq = Counter(bi_grams)


# Visualizing results for bigrams
top_bi_grams = bi_gram_freq.most_common(20)
bi_gram_strings, bi_gram_counts = zip(*[(f"{word1} {word2}", count) for (word1, word2), count in top_bi_grams])

plt.figure(figsize=(15,10))
plt.barh(bi_gram_strings, bi_gram_counts, color='skyblue')
plt.xlabel('Counts')
plt.title('Top bi-grams in Netflix Subreddit Past Month')
plt.gca().invert_yaxis()
plt.show()

# Print results for bi-grams
for bi_gram, count in top_bi_grams:
    print(f"{bi_gram[0]} {bi_gram[1]}: {count}")

In [ ]:
import praw
import matplotlib.pyplot as plt
from textblob import TextBlob
import warnings

warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id="uTp5HipGLhBhPv1S7oxk8A",
    client_secret="AcT-azqNdIT3onTltclkX4Q3ISUDDQ",
    user_agent="mads",
)

# Fetch top 100 posts from the 'netflix' subreddit from the past month
subreddit = reddit.subreddit('netflix')
top_posts = subreddit.top('month', limit=100)

# Extract comments mentioning "Breaking Bad"
breaking_bad_comments = []

for post in top_posts:
    for comment in post.comments:
        if isinstance(comment, praw.models.MoreComments):
            continue
        if "breaking bad" in comment.body.lower():
            breaking_bad_comments.append(comment.body)

# Compute sentiment for each comment
sentiments = [TextBlob(comment).sentiment.polarity for comment in breaking_bad_comments]

# Visualize sentiment distribution
plt.figure(figsize=(10,6))
plt.hist(sentiments, bins=30, color='skyblue', edgecolor='black')
plt.title('Sentiment Distribution for "Breaking Bad" Comments')
plt.xlabel('Polarity')
plt.ylabel('Number of Comments')
plt.show()